In [1]:
import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD

Intitializing Scala interpreter ...

Spark Web UI available at http://LAPTOP-0NFRQ5HD:4040
SparkContext available as 'sc' (version = 3.1.1, master = local[*], app id = local-1617962733014)
SparkSession available as 'spark'


import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD


In [164]:
val graph = GraphLoader.edgeListFile(sc,"data/soc-karate.mtx")
                .mapEdges(e => scala.util.Random.nextFloat)
                .mapVertices((vid,data) =>  false)
graph.vertices.collect

graph: org.apache.spark.graphx.Graph[Boolean,Float] = org.apache.spark.graphx.impl.GraphImpl@490eb9fe
res82: Array[(org.apache.spark.graphx.VertexId, Boolean)] = Array((34,false), (4,false), (16,false), (22,false), (28,false), (30,false), (14,false), (32,false), (24,false), (6,false), (8,false), (12,false), (18,false), (20,false), (26,false), (10,false), (2,false), (13,false), (19,false), (15,false), (21,false), (25,false), (29,false), (11,false), (27,false), (33,false), (23,false), (1,false), (17,false), (3,false), (7,false), (9,false), (31,false), (5,false))


In [33]:
def independant_cascade(graph:Graph[String,Float]):Graph[String,Float]={
    def mergeMsg (m1:String, m2:String):String = {
        if(m1=="become_inactive" || m2 == "become_inactive"){return "become_inactive"}
        if (m1 == "become_infected" || m2 == "become_infected"){return "become_infected"}
        return "nothing"
    }

    def vprog(VertexId:VertexId, VD:String, A:String): String ={
        if (VD == "susceptible" && A == "become_infected"){ return "infected"}
        if(VD=="infected" && A == "become_inactive"){return "inactive"}
        return VD
    }

    def sendMsg(triplet:EdgeTriplet[String, Float]): Iterator[(VertexId, String)]={
        if (triplet.srcAttr == "infected"){
            if (scala.util.Random.nextFloat < triplet.attr){
                return Iterator((triplet.dstId,"become_infected"),(triplet.srcId,"become_inactive"))
            }
        return Iterator((triplet.srcId,"become_inactive"))
        }
        return Iterator()
    }
    
    return graph.pregel("nothing",Int.MaxValue,EdgeDirection.Out)(vprog,sendMsg,mergeMsg)
    .mapVertices((vid,vdata) => if(vdata == "infected") "inactive" else vdata)
}

independant_cascade: (graph: org.apache.spark.graphx.Graph[String,Float])org.apache.spark.graphx.Graph[String,Float]


In [34]:
independant_cascade(graph).vertices.collect

res13: Array[(org.apache.spark.graphx.VertexId, String)] = Array((34,inactive), (4,susceptible), (16,susceptible), (22,susceptible), (28,susceptible), (30,susceptible), (14,inactive), (32,inactive), (24,inactive), (6,susceptible), (8,susceptible), (12,susceptible), (18,susceptible), (20,inactive), (26,inactive), (10,inactive), (2,inactive), (13,susceptible), (19,inactive), (15,susceptible), (21,inactive), (25,inactive), (29,inactive), (11,susceptible), (27,inactive), (33,inactive), (23,susceptible), (1,inactive), (17,susceptible), (3,inactive), (7,susceptible), (9,inactive), (31,susceptible), (5,susceptible))


In [48]:


type VD = (String,Float,Float)
type ED = Float
type Message = (String,Float)

def linear_threshhold(graph:Graph[VD,ED]):Graph[VD,ED]={
    def mergeMsg (m1:Message, m2:Message):Message = {
        if(m1._1=="become_inactive" || m2._1 == "become_inactive"){return ("become_inactive",0f)}
        if (m1._1 == "become_infected" && m2._1 == "become_infected"){return ("become_infected",m1._2+m2._2)}
        if (m1._1 == "become_infected" ){return m1}
        if (m2._1 == "become_infected" ){return m2}
        return ("nothing",0f)
    }

    def vprog(VertexId:VertexId, vdata:VD, m:Message): VD ={
        if (m._1 == "become_inactive"){
            return ("inactive", vdata._2, vdata._3)
        }
        if (m._1 == "become_infected"){
            val state = vdata._1
            val new_infection_rate = vdata._2 + m._2
            val threshold = vdata._3
            if (state == "susceptible" && new_infection_rate > threshold){
                return ("infected",new_infection_rate,threshold)
            }
            return (state,new_infection_rate,threshold)
        }
        return vdata
    }

    def sendMsg(triplet:EdgeTriplet[VD, ED]): Iterator[(VertexId, Message)]={
        if (triplet.srcAttr._1 == "infected"){
            return Iterator((triplet.dstId,("become_infected",triplet.attr)),(triplet.srcId,("become_inactive",0f)))

        return Iterator((triplet.srcId,("become_inactive",0f)))
        }
        return Iterator()
    }
    
    return graph.pregel(("nothing",0f),Int.MaxValue,EdgeDirection.Out)(vprog,sendMsg,mergeMsg)
}

defined type alias VD
defined type alias ED
defined type alias Message
linear_threshhold: (graph: org.apache.spark.graphx.Graph[VD,ED])org.apache.spark.graphx.Graph[VD,ED]


In [107]:
def normalize_graph_weights[VD](graph:Graph[VD,Float])(implicit m: ClassManifest[VD]):Graph[VD,Float] = {
    val weight_sums_per_dst = graph.triplets.map(t => (t.dstId,t.attr))
                            .reduceByKey( _ + _)
    val newEdges = graph.edges.map(e => (e.dstId,e))
         .join(weight_sums_per_dst).map{ case ((dstId,(edge,weightSum)))=>Edge(edge.srcId,edge.dstId,edge.attr/weightSum)}
    return Graph(graph.vertices,newEdges)
}

val graph = GraphLoader.edgeListFile(sc,"data/soc-karate.mtx")
                .mapEdges(e => scala.util.Random.nextFloat)
                .mapVertices[VD]((vid,i) =>  if(scala.util.Random.nextFloat <0.1) ("infected",1f, scala.util.Random.nextFloat) else ("susceptible",0f, scala.util.Random.nextFloat))
graph.edges.collect

normalize_graph_weights: [VD](graph: org.apache.spark.graphx.Graph[VD,Float])(implicit m: ClassManifest[VD])org.apache.spark.graphx.Graph[VD,Float]
graph: org.apache.spark.graphx.Graph[VD,Float] = org.apache.spark.graphx.impl.GraphImpl@42ba2aa7
res60: Array[org.apache.spark.graphx.Edge[Float]] = Array(Edge(2,1,0.070907235), Edge(3,1,0.10024041), Edge(3,2,0.051920712), Edge(4,1,0.28428066), Edge(4,2,0.60360086), Edge(4,3,0.9406253), Edge(5,1,0.122498274), Edge(6,1,0.5875056), Edge(7,1,0.2996779), Edge(7,5,0.73628646), Edge(7,6,0.549187), Edge(8,1,0.7103159), Edge(8,2,0.10904366), Edge(8,3,0.020172894), Edge(8,4,0.32169902), Edge(9,1,0.1611849), Edge(9,3,0.6805909), Edge(10,3,0.2593537), Edge(11,1,0.6347268), Edge(11,5,0.33655518), Edge(11,6,0.15796006), Edge(12,1,0.6241273), Edge(13,1,...


In [109]:
normalize_graph_weights(graph).edges.collect

res62: Array[org.apache.spark.graphx.Edge[Float]] = Array(Edge(3,2,0.02067454), Edge(4,2,0.24035051), Edge(7,6,0.39217466), Edge(8,2,0.04342058), Edge(8,4,0.24146138), Edge(11,6,0.11279934), Edge(13,4,0.22865711), Edge(14,2,0.03924461), Edge(14,4,0.52988154), Edge(17,6,0.495026), Edge(18,2,0.23011978), Edge(20,2,0.1823213), Edge(22,2,0.15113403), Edge(26,24,0.29384622), Edge(28,24,0.19369376), Edge(30,24,0.3089109), Edge(31,2,0.09273456), Edge(32,26,1.0), Edge(33,16,0.44138572), Edge(33,24,0.16643305), Edge(33,30,0.5094223), Edge(33,32,0.5581677), Edge(34,10,1.0), Edge(34,14,1.0), Edge(34,16,0.55861425), Edge(34,20,1.0), Edge(34,24,0.0371161), Edge(34,28,1.0), Edge(34,30,0.49057773), Edge(34,32,0.4418323), Edge(34,34,1.0), Edge(2,1,0.012187461), Edge(3,1,0.017229216), Edge(4,1,0.0488618...


In [55]:
linear_threshhold(graph).vertices.filter(vdata => vdata._2._2 >0).collect

res19: Array[(org.apache.spark.graphx.VertexId, VD)] = Array((20,(inactive,1.0,0.30429024)), (2,(inactive,1.0555973,0.39165944)), (1,(infected,2.5511122,0.7237318)), (9,(susceptible,0.23637855,0.23980576)), (31,(inactive,1.0,0.49187386)), (5,(inactive,1.0,0.029999077)))


In [160]:
def selectVerticeWithMaxDegree(graph:Graph[Boolean,_]):Graph[Boolean,_] = {
    val maxDegreeVertice = graph.triplets.filter(triplet => !triplet.srcAttr && !triplet.dstAttr)
                             .map(triplet => (triplet.srcId,1))
                             .reduceByKey(_+_)
                             .reduce((v1,v2) => if(v1._2 >= v2._2 ) v1 else v2)._1
    return graph.mapVertices((vid, vdata) => vid == maxDegreeVertice || vdata)
}

val selectKVerticesWithMaxDegrees = (k:Int) => Function.chain(List.fill(k)(selectVerticeWithMaxDegree _ ))

selectVerticeWithMaxDegree: (graph: org.apache.spark.graphx.Graph[Boolean, _])org.apache.spark.graphx.Graph[Boolean, _]
selectKVerticesWithMaxDegrees: Int => (org.apache.spark.graphx.Graph[Boolean, _] => org.apache.spark.graphx.Graph[Boolean, _]) = $Lambda$3934/0x000000010137d040@6e12ec5f


In [170]:
selectKVerticesWithMaxDegrees (10) (graph).vertices.join(graph.outDegrees).collect

res88: Array[(org.apache.spark.graphx.VertexId, (Boolean, Int))] = Array((34,(true,18)), (4,(true,3)), (22,(true,2)), (28,(true,3)), (30,(false,2)), (14,(true,4)), (32,(true,4)), (6,(false,1)), (8,(true,4)), (12,(false,1)), (18,(false,2)), (20,(false,2)), (26,(false,2)), (10,(false,1)), (2,(false,1)), (13,(false,2)), (29,(false,1)), (11,(true,3)), (33,(true,11)), (17,(false,2)), (3,(false,2)), (7,(true,3)), (9,(false,2)), (31,(false,2)), (5,(false,1)))
